In [2]:
# ============================================================
# Walk-Forward LSTM Long/Short vs Daily $10 Buy&Hold (SPX)
# + Rolling 5Y Windows + 5Y Block Bootstrap Monte Carlo
# Cleaned + No Lookahead + Low-noise execution
# ============================================================

from __future__ import annotations

import os
import warnings
from dataclasses import dataclass
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

# --- Reduce TensorFlow log noise BEFORE importing TF ---
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "2")  # 0=all, 1=INFO, 2=WARNING, 3=ERROR

# --- Filter common warnings (keeps tracebacks for real errors) ---
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

import tensorflow as tf
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping


# ============================================================
# Reproducibility (best-effort)
# ============================================================
def set_seeds(seed: int = 42) -> None:
    np.random.seed(seed)
    tf.keras.utils.set_random_seed(seed)


# ============================================================
# 1) Data loading (CSV/XLS/XLSX)
# ============================================================
def load_price_file(path: str | Path) -> pd.DataFrame:
    """
    Loads a price file and returns a DataFrame with columns:
      Date (datetime64[ns]) and Close (float)
    Supports .csv, .xls, .xlsx.

    Tries common column name variations (case/space-insensitive).
    """
    path = Path(path)
    if not path.exists():
        raise FileNotFoundError(f"File not found: {path.resolve()}")

    suffix = path.suffix.lower()

    if suffix == ".csv":
        df = pd.read_csv(path)
    elif suffix in {".xls", ".xlsx"}:
        # For .xls, xlrd is typically required; for .xlsx, openpyxl is typical.
        # Pandas will pick an engine if available; we supply common ones as fallback.
        try:
            df = pd.read_excel(path, engine="xlrd" if suffix == ".xls" else "openpyxl")
        except Exception:
            df = pd.read_excel(path)  # fallback (lets pandas choose)
    else:
        raise ValueError(f"Unsupported file type: {suffix}")

    # Normalize column names
    df.columns = (
        df.columns.astype(str)
        .str.strip()
        .str.replace(r"\s+", " ", regex=True)
        .str.lower()
    )

    # Identify date column
    date_candidates = ["date", "datetime", "timestamp"]
    date_col = next((c for c in df.columns if c in date_candidates), None)
    if date_col is None:
        # If no date column exists, create a synthetic index-based one
        df["date"] = pd.RangeIndex(len(df))
        date_col = "date"

    # Identify close column
    close_candidates = ["close", "adj close", "adj_close", "price", "close price"]
    close_col = next((c for c in df.columns if c in close_candidates), None)
    if close_col is None:
        # fallback to first numeric column
        num_cols = df.select_dtypes(include=["number"]).columns.tolist()
        if not num_cols:
            raise ValueError("No numeric price column found.")
        close_col = num_cols[0]

    out = df[[date_col, close_col]].copy()
    out.rename(columns={date_col: "Date", close_col: "Close"}, inplace=True)

    # Parse Date if not numeric range index
    if out["Date"].dtype != "int64" and out["Date"].dtype != "float64":
        out["Date"] = pd.to_datetime(out["Date"], errors="coerce")
        out = out.dropna(subset=["Date"]).sort_values("Date").reset_index(drop=True)

    out["Close"] = pd.to_numeric(out["Close"], errors="coerce")
    out = out.dropna(subset=["Close"]).reset_index(drop=True)

    if len(out) < 500:
        raise ValueError("Not enough rows after cleaning. Need a reasonable history.")

    return out


# ============================================================
# 2) Feature engineering (close-only)
# ============================================================
def features_from_close(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # Returns
    df["ret_1d"] = df["Close"].pct_change()
    with np.errstate(divide="ignore", invalid="ignore"):
        df["log_ret_1d"] = np.log(df["Close"]).diff()

    # Multi-horizon returns + rolling vol
    for w in (5, 10, 20, 60):
        df[f"ret_{w}d"] = df["Close"].pct_change(w)
        df[f"vol_{w}"] = df["ret_1d"].rolling(w).std()

    # SMAs + slopes
    for w in (5, 10, 20, 50, 200):
        sma = df["Close"].rolling(w).mean()
        df[f"sma_{w}"] = sma
        df[f"sma_slope_{w}"] = sma.diff()

    # EMA/MACD
    ema12 = df["Close"].ewm(span=12, adjust=False).mean()
    ema26 = df["Close"].ewm(span=26, adjust=False).mean()
    macd = ema12 - ema26
    macd_sig = macd.ewm(span=9, adjust=False).mean()
    df["ema_12"] = ema12
    df["ema_26"] = ema26
    df["macd"] = macd
    df["macd_signal"] = macd_sig

    # RSI(14)
    delta = df["Close"].diff()
    up = np.where(delta > 0, delta, 0.0)
    down = np.where(delta < 0, -delta, 0.0)
    roll_up = pd.Series(up, index=df.index).rolling(14).mean()
    roll_down = pd.Series(down, index=df.index).rolling(14).mean()
    rs = roll_up / (roll_down + 1e-12)
    df["rsi_14"] = 100 - (100 / (1 + rs))

    # Bollinger %b-ish scaled
    bb_w = 20
    bb_ma = df["Close"].rolling(bb_w).mean()
    bb_std = df["Close"].rolling(bb_w).std()
    df["bb_perc_b"] = (df["Close"] - bb_ma) / (2 * (bb_std + 1e-12))

    # Z-scores
    for w in (20, 60):
        mu = df["Close"].rolling(w).mean()
        sd = df["Close"].rolling(w).std()
        df[f"close_z_{w}"] = (df["Close"] - mu) / (sd + 1e-12)

    # ROC
    for w in (5, 10, 20):
        df[f"roc_{w}"] = df["Close"].pct_change(w)

    # Calendar features (if Date is datetime)
    if np.issubdtype(df["Date"].dtype, np.datetime64):
        d = df["Date"]
        df["dow"] = d.dt.dayofweek
        df["dom"] = d.dt.day
        df["month"] = d.dt.month
        df["is_month_end"] = d.dt.is_month_end.astype(int)
        df["is_quarter_end"] = d.dt.is_quarter_end.astype(int)
        df["is_year_end"] = d.dt.is_year_end.astype(int)

    # Target: next-day return (t -> t+1)
    df["target_ret_1d"] = df["ret_1d"].shift(-1)

    return df


def add_regimes(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # Simple regime encodings
    trend_score = df["sma_slope_50"].rolling(10).mean() / (df["Close"].rolling(50).std() + 1e-12)
    df["trend_score"] = trend_score

    trend_cond = (trend_score > 0.05) & (df["macd"] > df["macd_signal"])
    mr_cond = (df["rsi_14"] < 30) | (df["rsi_14"] > 70)

    df["regime"] = np.select(
        [trend_cond, mr_cond],
        ["trend", "mean_reversion"],
        default="neutral",
    )

    df = pd.get_dummies(df, columns=["regime"], prefix="reg", dtype=int)
    return df


# ============================================================
# 3) Sequence builder (aligned, fast)
# ============================================================
def build_sequences_aligned(X: np.ndarray, y: np.ndarray, lookback: int) -> tuple[np.ndarray, np.ndarray]:
    """
    For each t >= lookback-1:
      X_seq = X[t-lookback+1 : t+1]
      y_seq = y[t]
    """
    if lookback < 2:
        raise ValueError("lookback must be >= 2")

    n = len(X)
    if n < lookback:
        return np.empty((0, lookback, X.shape[1])), np.empty((0,))

    # sliding windows along first axis
    try:
        from numpy.lib.stride_tricks import sliding_window_view
        Xw = sliding_window_view(X, window_shape=(lookback, X.shape[1]))[:, 0, :, :]
        yw = y[lookback - 1 :]
        return Xw.astype(np.float32), yw.astype(np.float32)
    except Exception:
        # fallback loop (still correct)
        X_seq = []
        y_seq = []
        for t in range(lookback - 1, n):
            X_seq.append(X[t - lookback + 1 : t + 1])
            y_seq.append(y[t])
        return np.asarray(X_seq, dtype=np.float32), np.asarray(y_seq, dtype=np.float32)


# ============================================================
# 4) Model builder
# ============================================================
def build_model(input_shape: tuple[int, int], seed: int = 42) -> tf.keras.Model:
    tf.keras.utils.set_random_seed(seed)

    model = Sequential(
        [
            Input(shape=input_shape),
            LSTM(64, return_sequences=True),
            Dropout(0.2),
            LSTM(32),
            Dropout(0.2),
            Dense(32, activation="relu"),
            Dense(1),
        ]
    )
    model.compile(optimizer="adam", loss="mse")
    return model


# ============================================================
# 5) Walk-forward backtest (NO LOOKAHEAD)
# ============================================================
@dataclass(frozen=True)
class WalkForwardConfig:
    lookback: int = 60
    initial_train_years: int = 10
    retrain_every_days: int = 252 * 5
    epochs: int = 8
    batch_size: int = 256
    patience: int = 3
    val_frac: float = 0.2
    seed: int = 42


def walk_forward_oos_returns(
    price_df: pd.DataFrame,
    cfg: WalkForwardConfig,
    transaction_cost_bps: float = 3,
    slippage_bps: float = 2,
) -> pd.DataFrame:
    """
    Output columns:
      Date, spx_ret_next, pred_next, signal, strategy_ret
    Returns correspond to next-day close-to-close (t -> t+1).
    """

    feat = add_regimes(features_from_close(price_df)).dropna().reset_index(drop=True)

    # Feature/target split
    exclude = {"Date", "target_ret_1d"}
    X_cols = [c for c in feat.columns if c not in exclude]
    X_raw = feat[X_cols].to_numpy(dtype=float)
    y = feat["target_ret_1d"].to_numpy(dtype=float)  # return t->t+1

    has_date = np.issubdtype(feat["Date"].dtype, np.datetime64)

    # Determine first trade index (t0)
    if has_date:
        start_date = feat["Date"].iloc[0]
        cutoff_date = start_date + pd.DateOffset(years=cfg.initial_train_years)
        eligible = feat.index[feat["Date"] < cutoff_date]
        if len(eligible) == 0:
            raise ValueError("Not enough data before initial_train_years cutoff.")
        first_trade_t = int(eligible.max())
    else:
        first_trade_t = int(cfg.initial_train_years * 252)

    # Need enough history for sequences
    if first_trade_t < cfg.lookback + 50:
        raise ValueError(
            f"Not enough initial history for lookback={cfg.lookback}. "
            "Reduce lookback or increase initial_train_years / provide more data."
        )

    # Costs
    cost = (transaction_cost_bps + slippage_bps) / 10000.0

    preds = np.full(len(feat), np.nan, dtype=float)
    signals = np.zeros(len(feat), dtype=float)
    strat_ret = np.full(len(feat), np.nan, dtype=float)

    model: tf.keras.Model | None = None
    scaler: StandardScaler | None = None
    next_retrain_t = first_trade_t

    # IMPORTANT:
    # At time t, you do NOT know y[t] yet (it's t->t+1 return).
    # So training must use y up to (t-1).
    for t in range(first_trade_t, len(feat)):
        # retrain periodically using info available up to t (targets up to t-1)
        if model is None or t >= next_retrain_t:
            train_end = t - 1
            if train_end < cfg.lookback:
                # not enough target history yet
                next_retrain_t = t + 1
                continue

            X_train_raw = X_raw[: train_end + 1]
            y_train = y[: train_end + 1]

            scaler = StandardScaler().fit(X_train_raw)
            X_train = scaler.transform(X_train_raw)

            X_seq, y_seq = build_sequences_aligned(X_train, y_train, cfg.lookback)
            if len(X_seq) < 50:
                next_retrain_t = t + 1
                continue

            val_size = max(1, int(len(X_seq) * cfg.val_frac))
            if len(X_seq) <= val_size:
                val_size = 1

            X_tr, y_tr = X_seq[:-val_size], y_seq[:-val_size]
            X_val, y_val = X_seq[-val_size:], y_seq[-val_size:]

            model = build_model((cfg.lookback, X_seq.shape[-1]), seed=cfg.seed)
            es = EarlyStopping(monitor="val_loss", patience=cfg.patience, restore_best_weights=True)

            model.fit(
                X_tr, y_tr,
                validation_data=(X_val, y_val),
                epochs=cfg.epochs,
                batch_size=cfg.batch_size,
                shuffle=False,
                verbose=0,
                callbacks=[es],
            )

            next_retrain_t = t + cfg.retrain_every_days

        # If we didn't manage to train, skip prediction
        if model is None or scaler is None:
            continue

        # Predict y[t] using features up to t
        X_scaled_all = scaler.transform(X_raw)  # scaler fixed until next retrain
        X_last = X_scaled_all[t - cfg.lookback + 1 : t + 1].reshape(1, cfg.lookback, -1)

        pred = float(model.predict(X_last, verbose=0).ravel()[0])
        preds[t] = pred

        sig = float(np.sign(pred))  # -1, 0, +1
        signals[t] = sig

        r_next = float(y[t])  # realized next-day return

        prev_sig = float(signals[t - 1]) if t > first_trade_t else 0.0
        pos_change = abs(sig - prev_sig)  # 0, 1, or 2

        strat_ret[t] = sig * r_next - pos_change * cost

    out = pd.DataFrame(
        {
            "Date": feat["Date"] if has_date else pd.RangeIndex(len(feat)),
            "spx_ret_next": y,
            "pred_next": preds,
            "signal": signals,
            "strategy_ret": strat_ret,
        }
    )

    # keep only where we are actually trading AND have strategy returns
    out = out.iloc[first_trade_t:].dropna(subset=["strategy_ret"]).reset_index(drop=True)
    return out


# ============================================================
# 6) Wealth evolution (daily contributions)
# ============================================================
def evolve_with_daily_contribution(r: np.ndarray, contrib: float = 10.0, v0: float = 0.0) -> np.ndarray:
    """
    V_{t+1} = (V_t + contrib) * (1 + r_t)
    """
    r = np.asarray(r, dtype=float)
    V = np.empty(len(r) + 1, dtype=float)
    V[0] = float(v0)
    for t in range(len(r)):
        V[t + 1] = (V[t] + contrib) * (1.0 + r[t])
    return V


def max_drawdown(wealth: np.ndarray) -> float:
    wealth = np.asarray(wealth, dtype=float)
    peaks = np.maximum.accumulate(wealth)
    dd = (wealth - peaks) / (peaks + 1e-12)
    return float(dd.min())


def approx_sharpe(daily_r: np.ndarray) -> float:
    daily_r = np.asarray(daily_r, dtype=float)
    mu = np.nanmean(daily_r)
    sd = np.nanstd(daily_r) + 1e-12
    return float(np.sqrt(252) * mu / sd)


def contribution_multiple(final_wealth: float, contrib: float, n_days: int) -> float:
    total = contrib * n_days
    return float(final_wealth / (total + 1e-12))


def xirr_daily(dates: pd.Series, contrib: float, ending_wealth: float) -> float:
    """
    Money-weighted return (IRR) for daily contributions.
    Cashflows: -contrib each day; final +ending_wealth at last date.
    """
    dates = pd.to_datetime(pd.Series(dates)).reset_index(drop=True)

    cf_dates = list(dates)
    cfs = [-contrib] * len(dates)
    cf_dates.append(dates.iloc[-1])
    cfs.append(float(ending_wealth))

    t0 = cf_dates[0]
    yearfrac = np.array([(d - t0).days / 365.25 for d in cf_dates], dtype=float)
    cfs = np.array(cfs, dtype=float)

    def xnpv(rate: float) -> float:
        return float(np.sum(cfs / (1.0 + rate) ** yearfrac))

    def dxnpv(rate: float) -> float:
        return float(np.sum(-yearfrac * cfs / (1.0 + rate) ** (yearfrac + 1.0)))

    r = 0.08
    for _ in range(100):
        f = xnpv(r)
        df = dxnpv(r)
        if abs(df) < 1e-12:
            break
        r_new = r - f / df
        r_new = float(np.clip(r_new, -0.95, 5.0))
        if abs(r_new - r) < 1e-10:
            r = r_new
            break
        r = r_new
    return float(r)


# ============================================================
# 7) Rolling 5Y windows
# ============================================================
def rolling_windows_5y(df_ret: pd.DataFrame, block_years: int = 5, contrib: float = 10.0) -> pd.DataFrame:
    out = []
    dates = df_ret["Date"]
    has_date = np.issubdtype(pd.Series(dates).dtype, np.datetime64)

    if has_date:
        dates = pd.to_datetime(dates).reset_index(drop=True)

        for start_i in range(len(df_ret)):
            start_date = dates.iloc[start_i]
            end_date = start_date + pd.DateOffset(years=block_years)

            end_i = int(dates.searchsorted(end_date, side="left") - 1)
            if end_i <= start_i:
                continue

            # avoid truncated tail windows
            if (dates.iloc[end_i] - start_date).days < int(365.25 * block_years * 0.95):
                continue

            r_bh = df_ret["spx_ret_next"].iloc[start_i : end_i + 1].to_numpy()
            r_ls = df_ret["strategy_ret"].iloc[start_i : end_i + 1].to_numpy()
            d_win = dates.iloc[start_i : end_i + 1]

            W_bh = evolve_with_daily_contribution(r_bh, contrib=contrib)
            W_ls = evolve_with_daily_contribution(r_ls, contrib=contrib)

            out.append(
                {
                    "start": start_date,
                    "end": dates.iloc[end_i],
                    "days": int(len(r_bh)),
                    "final_bh": float(W_bh[-1]),
                    "final_ls": float(W_ls[-1]),
                    "mult_bh": contribution_multiple(W_bh[-1], contrib, len(r_bh)),
                    "mult_ls": contribution_multiple(W_ls[-1], contrib, len(r_ls)),
                    "sharpe_bh": approx_sharpe(r_bh),
                    "sharpe_ls": approx_sharpe(r_ls),
                    "mdd_bh": max_drawdown(W_bh),
                    "mdd_ls": max_drawdown(W_ls),
                    "xirr_bh": xirr_daily(d_win, contrib=contrib, ending_wealth=W_bh[-1]),
                    "xirr_ls": xirr_daily(d_win, contrib=contrib, ending_wealth=W_ls[-1]),
                }
            )
    else:
        L = int(252 * block_years)
        for start_i in range(0, len(df_ret) - L + 1):
            end_i = start_i + L - 1
            r_bh = df_ret["spx_ret_next"].iloc[start_i : end_i + 1].to_numpy()
            r_ls = df_ret["strategy_ret"].iloc[start_i : end_i + 1].to_numpy()

            W_bh = evolve_with_daily_contribution(r_bh, contrib=contrib)
            W_ls = evolve_with_daily_contribution(r_ls, contrib=contrib)

            out.append(
                {
                    "start": start_i,
                    "end": end_i,
                    "days": L,
                    "final_bh": float(W_bh[-1]),
                    "final_ls": float(W_ls[-1]),
                    "mult_bh": contribution_multiple(W_bh[-1], contrib, L),
                    "mult_ls": contribution_multiple(W_ls[-1], contrib, L),
                    "sharpe_bh": approx_sharpe(r_bh),
                    "sharpe_ls": approx_sharpe(r_ls),
                    "mdd_bh": max_drawdown(W_bh),
                    "mdd_ls": max_drawdown(W_ls),
                }
            )

    return pd.DataFrame(out)


# ============================================================
# 8) 5Y block bootstrap Monte Carlo
# ============================================================
def bootstrap_5y_blocks(
    df_ret: pd.DataFrame,
    n_sims: int = 5000,
    block_years: int = 5,
    contrib: float = 10.0,
    seed: int = 123,
) -> pd.DataFrame:
    windows = rolling_windows_5y(df_ret, block_years=block_years, contrib=contrib)
    if windows.empty:
        raise ValueError("No valid 5-year windows found. Check data length and dates.")

    rng = np.random.default_rng(seed)
    sampled_idx = rng.integers(0, len(windows), size=n_sims)
    sim = windows.iloc[sampled_idx].copy().reset_index(drop=True)

    sim["sim"] = np.arange(n_sims)
    sim["ls_minus_bh"] = sim["final_ls"] - sim["final_bh"]
    sim["ls_over_bh"] = sim["final_ls"] / (sim["final_bh"] + 1e-12)
    return sim


# ============================================================
# 9) Plot helpers
# ============================================================
def plot_wealth_paths(df_ret: pd.DataFrame, W_bh: np.ndarray, W_ls: np.ndarray, outpath: Path) -> None:
    plt.figure(figsize=(12, 6))
    x = pd.to_datetime(df_ret["Date"], errors="coerce")
    if x.isna().all():
        x = np.arange(len(df_ret))

    plt.plot(x, W_bh[1:], label="Buy & Hold (daily $10)", color="steelblue")
    plt.plot(x, W_ls[1:], label="Long/Short LSTM (daily $10)", color="tomato", alpha=0.9)
    plt.title("Wealth Paths with Daily $10 Contributions")
    plt.ylabel("Portfolio Value ($)")
    plt.xlabel("Date" if not isinstance(x, np.ndarray) else "Index")
    plt.legend()
    plt.tight_layout()
    plt.savefig(outpath, dpi=150)
    plt.close()


def plot_mc_distribution(mc5: pd.DataFrame, out1: Path, out2: Path) -> None:
    plt.figure(figsize=(12, 5))
    plt.hist(mc5["final_bh"], bins=60, alpha=0.55, label="BH final wealth (5y)", color="steelblue")
    plt.hist(mc5["final_ls"], bins=60, alpha=0.55, label="LS final wealth (5y)", color="tomato")
    plt.title("5-Year Block Bootstrap: Distribution of Final Wealth")
    plt.xlabel("Final wealth over 5y ($, daily $10 contributions)")
    plt.ylabel("Frequency")
    plt.legend()
    plt.tight_layout()
    plt.savefig(out1, dpi=150)
    plt.close()

    plt.figure(figsize=(12, 5))
    plt.hist(mc5["ls_minus_bh"], bins=80, alpha=0.9, color="purple")
    plt.axvline(0, color="black", linewidth=2)
    plt.title("5-Year Block Bootstrap: (LS - BH) Final Wealth")
    plt.xlabel("Final wealth difference ($)")
    plt.ylabel("Frequency")
    plt.tight_layout()
    plt.savefig(out2, dpi=150)
    plt.close()


# ============================================================
# 10) Main
# ============================================================
def main() -> None:
    set_seeds(42)

    # ---- Load ----
    price = load_price_file("spx.csv")

    # ---- Walk-forward LSTM OOS strategy returns ----
    cfg = WalkForwardConfig(
        lookback=60,
        initial_train_years=10,
        retrain_every_days=252 * 5,
        epochs=8,
        batch_size=256,
        patience=3,
        val_frac=0.2,
        seed=42,
    )

    df_ret = walk_forward_oos_returns(
        price_df=price,
        cfg=cfg,
        transaction_cost_bps=3,
        slippage_bps=2,
    )

    outdir = Path(".")
    df_ret.to_csv(outdir / "oos_daily_returns.csv", index=False)

    # ---- Full-period analytics (daily $10 contributions) ----
    contrib = 10.0
    r_bh = df_ret["spx_ret_next"].to_numpy(dtype=float)
    r_ls = df_ret["strategy_ret"].to_numpy(dtype=float)

    W_bh = evolve_with_daily_contribution(r_bh, contrib=contrib)
    W_ls = evolve_with_daily_contribution(r_ls, contrib=contrib)

    summary = {
        "days_traded": len(df_ret),
        "period_start": df_ret["Date"].iloc[0],
        "period_end": df_ret["Date"].iloc[-1],
        "final_wealth_bh": float(W_bh[-1]),
        "final_wealth_ls": float(W_ls[-1]),
        "contrib_multiple_bh": contribution_multiple(W_bh[-1], contrib, len(df_ret)),
        "contrib_multiple_ls": contribution_multiple(W_ls[-1], contrib, len(df_ret)),
        "sharpe_bh_approx": approx_sharpe(r_bh),
        "sharpe_ls_approx": approx_sharpe(r_ls),
        "max_drawdown_bh": max_drawdown(W_bh),
        "max_drawdown_ls": max_drawdown(W_ls),
    }

    # XIRR only if dates are real
    try:
        pd.to_datetime(df_ret["Date"])
        summary["xirr_bh"] = xirr_daily(df_ret["Date"], contrib=contrib, ending_wealth=W_bh[-1])
        summary["xirr_ls"] = xirr_daily(df_ret["Date"], contrib=contrib, ending_wealth=W_ls[-1])
    except Exception:
        pass

    print("\n=== Full-period descriptive summary (daily $10 contributions) ===")
    for k, v in summary.items():
        print(f"{k}: {v}")

    (outdir / "summary_full_period.txt").write_text(
        "\n".join([f"{k}: {v}" for k, v in summary.items()]) + "\n",
        encoding="utf-8",
    )

    wealth_df = pd.DataFrame(
        {
            "Date": df_ret["Date"],
            "wealth_bh": W_bh[1:],
            "wealth_ls": W_ls[1:],
        }
    )
    wealth_df.to_csv(outdir / "wealth_paths_full_period.csv", index=False)

    plot_wealth_paths(df_ret, W_bh, W_ls, outdir / "figure_wealth_paths.png")

    # ---- Rolling 5-year windows ----
    roll5 = rolling_windows_5y(df_ret, block_years=5, contrib=contrib)
    roll5.to_csv(outdir / "rolling_5y_metrics.csv", index=False)
    print(f"\nComputed {len(roll5)} rolling 5-year windows -> rolling_5y_metrics.csv")

    # ---- 5-year block bootstrap Monte Carlo ----
    mc5 = bootstrap_5y_blocks(df_ret, n_sims=5000, block_years=5, contrib=contrib, seed=123)
    mc5.to_csv(outdir / "mc_5y_block_bootstrap.csv", index=False)

    p_win = float(np.mean(mc5["final_ls"] > mc5["final_bh"]))
    print("\n=== 5-year block bootstrap MC summary ===")
    print(f"P(LS beats BH over a random 5y block): {p_win:.3f}")
    print(f"Median LS/BH: {np.median(mc5['ls_over_bh']):.3f}")
    print(f"Median (LS - BH) wealth: {np.median(mc5['ls_minus_bh']):.2f}")
    print(
        "5th/95th pct (LS - BH): "
        f"{np.quantile(mc5['ls_minus_bh'], 0.05):.2f} / {np.quantile(mc5['ls_minus_bh'], 0.95):.2f}"
    )

    plot_mc_distribution(
        mc5,
        outdir / "figure_mc5_final_wealth.png",
        outdir / "figure_mc5_ls_minus_bh.png",
    )

    print("\nOutputs written:")
    for f in [
        "oos_daily_returns.csv",
        "wealth_paths_full_period.csv",
        "rolling_5y_metrics.csv",
        "mc_5y_block_bootstrap.csv",
        "summary_full_period.txt",
        "figure_wealth_paths.png",
        "figure_mc5_final_wealth.png",
        "figure_mc5_ls_minus_bh.png",
    ]:
        print(f"- {f}")


if __name__ == "__main__":
    main()


=== Full-period descriptive summary (daily $10 contributions) ===
days_traded: 5463
period_start: 1996-10-15 00:00:00
period_end: 2018-06-28 00:00:00
final_wealth_bh: 115533.91818780858
final_wealth_ls: 32137.25750646982
contrib_multiple_bh: 2.1148438255136113
contrib_multiple_ls: 0.5882712338727772
sharpe_bh_approx: 0.42267102425628184
sharpe_ls_approx: -0.41412583306405076
max_drawdown_bh: -0.513651333143571
max_drawdown_ls: -0.40356450728832544
xirr_bh: 0.06412003001568187
xirr_ls: 5.0

Computed 4267 rolling 5-year windows -> rolling_5y_metrics.csv

=== 5-year block bootstrap MC summary ===
P(LS beats BH over a random 5y block): 0.144
Median LS/BH: 0.711
Median (LS - BH) wealth: -3879.97
5th/95th pct (LS - BH): -6842.39 / 6460.09

Outputs written:
- oos_daily_returns.csv
- wealth_paths_full_period.csv
- rolling_5y_metrics.csv
- mc_5y_block_bootstrap.csv
- summary_full_period.txt
- figure_wealth_paths.png
- figure_mc5_final_wealth.png
- figure_mc5_ls_minus_bh.png
